# Inequality Metrics for Sponsorship and Earnings

This notebook quantifies inequality in sponsorship and earnings by computing:

- Gini coefficients for `total_sponsors` and `estimated_earnings`
- Share of sponsorships / earnings captured by the top X% of accounts

In [ ]:
import numpy as np
import pandas as pd
from data_loader import load_sample_data

In [ ]:
def gini(values) -> float:
    arr = np.array(list(values), dtype=float)
    arr = arr[~np.isnan(arr)]
    if arr.size == 0:
        return float("nan")
    if np.all(arr == 0):
        return 0.0
    arr = np.sort(arr)
    n = arr.size
    cum = np.cumsum(arr)
    g = (2.0 * np.sum((np.arange(1, n + 1) * arr)) - (n + 1) * cum[-1]) / (n * cum[-1])
    return float(g)


def top_share(values, top_fraction: float) -> float:
    arr = np.array(list(values), dtype=float)
    arr = arr[~np.isnan(arr)]
    if arr.size == 0:
        return float("nan")
    if top_fraction <= 0 or top_fraction >= 1:
        raise ValueError("top_fraction must be in (0, 1).")
    n = arr.size
    k = max(1, int(round(top_fraction * n)))
    arr_sorted = np.sort(arr)[::-1]
    top_sum = arr_sorted[:k].sum()
    total = arr_sorted.sum() or 1.0
    return float(top_sum / total)


def compute_inequality_metrics(df: pd.DataFrame) -> pd.DataFrame:
    metrics = []
    for var in ["total_sponsors", "estimated_earnings"]:
        if var not in df.columns:
            continue
        series = df[var]
        metrics.append(
            {
                "variable": var,
                "gini": gini(series),
                "top_1pct_share": top_share(series, 0.01),
                "top_10pct_share": top_share(series, 0.10),
            }
        )
    return pd.DataFrame(metrics)

## Load Data and Compute Inequality Metrics

In [ ]:
df = load_sample_data()
metrics = compute_inequality_metrics(df)

## Display Results

In [ ]:
if metrics.empty:
    print("No variables available for inequality analysis.")
else:
    print("\nInequality metrics for sponsorship and earnings:\n")
    metrics